In [10]:
#Import needed packages
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD
from torch.autograd import Variable
import numpy as np
from sklearn.model_selection import train_test_split
import torch.utils.data as utils
import pdb

class Unit(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(Unit,self).__init__()
        

        self.conv = nn.Conv2d(in_channels=in_channels,kernel_size=3,out_channels=out_channels,stride=1,padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()

    def forward(self,input):
        output = self.conv(input)
        output = self.bn(output)
        output = self.relu(output)

        return output

class SimpleNet(nn.Module):
    def __init__(self,num_classes=5):
        super(SimpleNet,self).__init__()

        #Create 14 layers of the unit with max pooling in between
        self.unit1 = Unit(in_channels=3,out_channels=32)
        self.unit2 = Unit(in_channels=32, out_channels=32)
        self.unit3 = Unit(in_channels=32, out_channels=32)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.unit4 = Unit(in_channels=32, out_channels=64)
        self.unit5 = Unit(in_channels=64, out_channels=64)
        self.unit6 = Unit(in_channels=64, out_channels=64)
        self.unit7 = Unit(in_channels=64, out_channels=64)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.unit8 = Unit(in_channels=64, out_channels=128)
        self.unit9 = Unit(in_channels=128, out_channels=128)
        self.unit10 = Unit(in_channels=128, out_channels=128)
        self.unit11 = Unit(in_channels=128, out_channels=128)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.unit12 = Unit(in_channels=128, out_channels=128)
        self.unit13 = Unit(in_channels=128, out_channels=128)
        self.unit14 = Unit(in_channels=128, out_channels=128)

        self.avgpool = nn.AvgPool2d(kernel_size=4)
        
        #Add all the units into the Sequential layer in exact order
        self.net = nn.Sequential(self.unit1, self.unit2, self.unit3, self.pool1, self.unit4, self.unit5, self.unit6
                                 ,self.unit7, self.pool2, self.unit8, self.unit9, self.unit10, self.unit11, self.pool3,
                                 self.unit12, self.unit13, self.unit14, self.avgpool)

        self.fc = nn.Linear(in_features=128,out_features=num_classes)

    def forward(self, input):
        output = self.net(input)

        output = output.view(-1,128)
        output = self.fc(output)
        return output

#Define transformations for the training set, flip the images randomly, crop out and apply mean and std normalization
train_transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 32

#Load the training set
flower_imgs = np.squeeze(np.load('Flowers/flower_imgs.npy'))
flower_labels = np.squeeze(np.load('Flowers/flower_labels.npy'))

X,y = flower_imgs,flower_labels

# Splitting Dataset
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0)

class_names = ['daisy', 'dandelion', 'rose', 'sunflower', 
               'tulip']
# pdb.set_trace()
# Preprocess the data : Scale these values to a range of 0 to 1
x_train = np.transpose(x_train, (0,3,1,2))
x_test = np.transpose(x_test, (0,3,1,2))
# x_train = x_train.reshape(x_train.shape[0], 3, 32,32)
# x_test = x_test.reshape(x_test.shape[0], 3, 32, 32)
input_shape = (32, 32, 3)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
#Create a loader for the training set
# tensor_x = torch.tensor(x_train)
tensor_x = torch.from_numpy(x_train)
tensor_y = torch.from_numpy(y_train)
# tensor_y = torch.tensor(y_train)
train_set = utils.TensorDataset(tensor_x,tensor_y)
train_loader = DataLoader(train_set,batch_size=32,shuffle=True,num_workers=4)

#Define transformations for the test set
test_transformations = transforms.Compose([
   transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))

])

#Load the test set, note that train is set to False
tensor_x = torch.from_numpy(x_test)
tensor_y = torch.from_numpy(y_test)
# tensor_x = torch.tensor(x_test)
# tensor_y = torch.tensor(y_test)
test_set = utils.TensorDataset(tensor_x,tensor_y)
test_loader = DataLoader(test_set,batch_size=32,shuffle=True,num_workers=4)


#Check if gpu support is available
cuda_avail = torch.cuda.is_available()
# cuda_avail = False
#Create model, optimizer and loss function
model = SimpleNet(num_classes=5)

if cuda_avail:
    model.cuda()

optimizer = SGD(model.parameters(), lr=0.001,weight_decay=5e-4,momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

#Create a learning rate adjustment function that divides the learning rate by 10 every 30 epochs
def adjust_learning_rate(epoch):

    lr = 0.001

    #if epoch > 180:
     #   lr = lr / 1000000
    if epoch > 150:
        lr = lr / 100
    #elif epoch > 120:
     #   lr = lr / 10000
    elif epoch > 90:
        lr = lr / 10
    #elif epoch > 60:
     #   lr = lr / 100
    #elif epoch > 30:
     #   lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr



def test():
    model.eval()
    test_acc = 0.0
    for i, (images, labels) in enumerate(test_loader):

        if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

        #Predict classes using images from the test set
        outputs = model(images)
        _,prediction = torch.max(outputs.data, 1)
        prediction = prediction.data
        test_acc = (i*test_acc +
                    torch.mean(prediction.eq(labels.data).float()))/(i+1)



    #Compute the average acc and loss over all 10000 test images


    return test_acc

def train(num_epochs):
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            #Move images and labels to gpu if available
            if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            #Clear all accumulated gradients
            optimizer.zero_grad()
            #Predict classes using images from the test set
            outputs = model(images)
            #Compute the loss based on the predictions and actual labels
            loss = loss_fn(outputs,labels)
            #Backpropagate the loss
            loss.backward()

            #Adjust parameters according to the computed gradients
            optimizer.step()

            train_loss = (train_loss*i + loss.cpu().data[0])/(i+1)
            # pdb.set_trace()

            _, prediction = torch.max(outputs.data, 1)

            train_acc = (i*train_acc +
                         torch.mean(prediction.eq(labels.data).float()))/(i+1)

        #Call the learning rate adjustment function
        adjust_learning_rate(epoch)

        #Compute the average acc and loss over all 50000 training images
        # train_acc = train_acc / 50000.
        # train_loss = train_loss / 50000.

        #Evaluate on the test set
        test_acc = test()


        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch, train_acc, train_loss,test_acc))


if __name__ == "__main__":
    train(200)

TypeError: argument 0 is not a Variable

In [12]:
type(tensor_y)

torch.LongTensor